In [ ]:
# Credential Scheme Benchmark Analysis
import json
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import datetime

# Configuration settings
BASE_DIR = Path("../target/criterion")
OUTPUT_DIR = Path("./benchmark_analysis")

# Scheme configuration
SCHEME_DIRS = {
    "bbs_plus_og_anoncreds": "bbs_plus_og",
    "ps_anoncreds": "ps",
    "bbs_plus_16_anoncreds": "bbs_plus_16",
    "ps_utt_anoncreds_std": "ps_utt_std",
    "ps_utt_anoncreds_imp": "ps_utt_imp"
}

# Display names for schemes (in desired order)
SCHEME_NAMES = {
    "bbs_plus_og": "BBS+ 06",
    "ps": "PS 16",
    "bbs_plus_16": "BBS+ 16",
    "ps_utt_std": "PS-UTT G1",
    "ps_utt_imp": "PS-UTT G2"
}

# Ordered list of schemes for consistent visualization
SCHEME_ORDER = [
    "bbs_plus_og",
    "bbs_plus_16",
    "ps",
    "ps_utt_std",
    "ps_utt_imp"
]

# Focus only on the requested operations
OPERATIONS = ["obtain", "issue", "show", "verify"]

# Custom color palette grouping schemes by family
SCHEME_COLORS = {
    "bbs_plus_og": "#4169E1",  # Royal Blue for BBS+ OG
    "bbs_plus_16": "#87CEFA",  # Light Sky Blue for BBS+ 16
    "ps": "#FF8C00",          # Dark Orange for PS Standard
    "ps_utt_std": "#CD5C5C",  # Indian Red for PS-UTT Standard
    "ps_utt_imp": "#E9967A",  # Dark Salmon for PS-UTT Improved
}

# Use our custom scheme colors instead of the default palette
COLORS = [SCHEME_COLORS[scheme] for scheme in SCHEME_ORDER]

# Visualization settings
MESSAGE_SIZES_TO_VISUALIZE = [5, 10, 20, 30]

# Set up visualization style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context("paper", font_scale=1.5)

print("Setup complete!")

In [ ]:
def extract_benchmark_data(base_dir: Path) -> pd.DataFrame:
    """Extract benchmark data from Criterion output directories."""
    print(f"Extracting benchmark data from {base_dir}")

    all_data = []

    # Process each scheme directory
    for scheme_dir in base_dir.iterdir():
        if not scheme_dir.is_dir():
            continue

        scheme_key = scheme_dir.name
        if scheme_key not in SCHEME_DIRS:
            print(f"Skipping unknown directory: {scheme_key}")
            continue

        # Map directory name to scheme identifier
        scheme = SCHEME_DIRS[scheme_key]
        print(f"Processing scheme: {SCHEME_NAMES.get(scheme, scheme)}")

        # Process each benchmark directory
        for bench_dir in scheme_dir.iterdir():
            if not bench_dir.is_dir() or bench_dir.name == "report":
                continue

            try:
                # Parse benchmark directory name
                bench_name = bench_dir.name

                # Try both formats: operation_messages_N and scheme_operation_messages_N
                match = re.search(r'(?:[a-z_]+_)?([a-z]+)_messages_(\d+)', bench_name)
                if not match:
                    print(f"  Skipping {bench_name}: Unable to parse benchmark name format")
                    continue

                operation = match.group(1)  # operation name
                msg_size = int(match.group(2))

                # Only include operations we're interested in
                if operation not in OPERATIONS:
                    continue

                # Find and load the estimates.json file
                estimates_file = bench_dir / "new" / "estimates.json"
                if not estimates_file.exists():
                    print(f"  Skipping {bench_name}: No estimates file found")
                    continue

                with open(estimates_file, 'r') as f:
                    data = json.load(f)

                # Extract key metrics (converting from ns to ms)
                mean_time = data["mean"]["point_estimate"] / 1_000_000
                std_dev = data["mean"]["standard_error"] / 1_000_000
                median = data["median"]["point_estimate"] / 1_000_000
                min_time = data["slope"]["point_estimate"] / 1_000_000  # Best case

                all_data.append({
                    "scheme": scheme,
                    "display_name": SCHEME_NAMES.get(scheme, scheme),
                    "operation": operation,
                    "msg_size": msg_size,
                    "mean_ms": mean_time,
                    "median_ms": median,
                    "std_dev_ms": std_dev,
                    "min_ms": min_time
                })

                print(f"  Processed: {scheme_dir.name}/{bench_name} - {mean_time:.2f} ms")

            except Exception as e:
                print(f"  Error processing {bench_dir.name}: {e}")

    if not all_data:
        raise ValueError("No valid benchmark data found!")

    # Convert to DataFrame and apply basic cleaning
    df = pd.DataFrame(all_data)

    # Add scheme_order column based on SCHEME_ORDER for consistent sorting
    order_map = {scheme: i for i, scheme in enumerate(SCHEME_ORDER)}
    df['scheme_order'] = df['scheme'].map(lambda x: order_map.get(x, 999))

    # Ensure operations are in standard order for visualization
    op_order = {op: i for i, op in enumerate(OPERATIONS)}
    df['op_order'] = df['operation'].map(lambda x: op_order.get(x, 999))

    # Sort by scheme order, msg_size, and operation order
    df = df.sort_values(['scheme_order', 'msg_size', 'op_order'])

    # Drop sorting columns
    df = df.drop(columns=['scheme_order', 'op_order'])

    return df

def save_to_csv(df, base_dir="z_py_tests"):
    """Save the benchmark data to a CSV file with date-time in the filename."""
    # Create directory if it doesn't exist
    csv_dir = Path(base_dir)
    csv_dir.mkdir(parents=True, exist_ok=True)
    
    # Generate filename with current date and time
    now = datetime.datetime.now()
    date_time_str = now.strftime("%Y%m%d_%H%M%S")
    filename = f"anoncred_analysis_{date_time_str}.csv"
    
    # Full path to save the CSV
    csv_path = csv_dir / filename
    
    # Save to CSV
    df.to_csv(csv_path, index=False)
    
    print(f"Saved benchmark data to {csv_path}")
    return csv_path

# Execute the extraction and CSV saving
benchmark_df = extract_benchmark_data(BASE_DIR)
csv_path = save_to_csv(benchmark_df)
print(f"Data extraction complete. CSV saved to: {csv_path}")

In [ ]:
def create_performance_comparison_charts(df: pd.DataFrame):
    """Create performance comparison graphs for specified message sizes with increased spacing."""
    # Get available schemes in the preferred order
    available_schemes = [s for s in SCHEME_ORDER if s in df['scheme'].unique()]

    # Create figures for each message size
    figures = {}

    for msg_size in MESSAGE_SIZES_TO_VISUALIZE:
        msg_data = df[df['msg_size'] == msg_size]
        if msg_data.empty:
            print(f"No data for {msg_size} messages, skipping...")
            continue

        # Filter operations that exist in the data
        available_ops = msg_data['operation'].unique()
        ops_to_use = [op for op in OPERATIONS if op in available_ops]

        fig, ax = plt.subplots(figsize=(12, 7))

        bar_width = 0.25  # Narrower width to accommodate more schemes
        group_spacing = 0.4  # Add spacing between operation groups

        # Create wider spacing between operation groups
        index = np.arange(len(ops_to_use)) * (1 + group_spacing)

        for i, scheme in enumerate(available_schemes):
            scheme_data = msg_data[msg_data['scheme'] == scheme]
            if not scheme_data.empty:
                # Ensure data is ordered by operation
                op_means = []
                for op in ops_to_use:
                    op_mean = scheme_data[scheme_data['operation'] == op]['mean_ms'].values
                    op_means.append(op_mean[0] if len(op_mean) > 0 else 0)

                # Use scheme-specific color instead of palette index
                ax.bar(
                    index + i * bar_width,
                    op_means,
                    bar_width,
                    label=SCHEME_NAMES.get(scheme, scheme),
                    color=SCHEME_COLORS[scheme]  # Use scheme-specific color
                )

        ax.set_xlabel('Operation')
        ax.set_ylabel('Time (ms)')
        ax.set_title(f'Performance Comparison at {msg_size} Messages')
        ax.set_xticks(index + (len(available_schemes) - 1) * bar_width / 2)
        ax.set_xticklabels([op.capitalize() for op in ops_to_use])
        ax.legend()

        # Add gridlines for better readability
        ax.grid(axis='y', linestyle='--', alpha=0.7)

        plt.tight_layout()
        figures[msg_size] = fig

        print(f"Created performance comparison chart for {msg_size} messages")

    return figures

def create_operation_performance_charts(df: pd.DataFrame):
    """Create line charts showing performance by message count for each operation."""
    # Get available schemes in the preferred order
    available_schemes = [s for s in SCHEME_ORDER if s in df['scheme'].unique()]

    # Create a figure for each operation
    figures = {}

    for operation in OPERATIONS:
        # Filter data for this operation
        op_data = df[df['operation'] == operation]
        if op_data.empty:
            print(f"No data for operation {operation}, skipping...")
            continue

        # Create a figure
        fig, ax = plt.subplots(figsize=(12, 7))

        # Plot a line for each scheme
        for i, scheme in enumerate(available_schemes):
            scheme_data = op_data[op_data['scheme'] == scheme]
            if not scheme_data.empty:
                # Sort by message size
                scheme_data = scheme_data.sort_values('msg_size')

                # Plot the line using scheme-specific color
                ax.plot(
                    scheme_data['msg_size'],
                    scheme_data['mean_ms'],
                    'o-',
                    label=SCHEME_NAMES.get(scheme, scheme),
                    color=SCHEME_COLORS[scheme]  # Use scheme-specific color
                )

        ax.set_xlabel('Number of Messages')
        ax.set_ylabel('Time (ms)')
        ax.set_title(f'{operation.capitalize()} Operation Performance')
        ax.grid(True, linestyle='--', alpha=0.7)
        ax.legend()

        # Ensure x-axis shows only the message sizes we have
        ax.set_xticks(sorted(df['msg_size'].unique()))

        plt.tight_layout()
        figures[operation] = fig

        print(f"Created performance chart for {operation} operation")

    return figures

# Execute chart creation (using the DataFrame from the previous cell)
try:
    comparison_charts = create_performance_comparison_charts(benchmark_df)
    operation_charts = create_operation_performance_charts(benchmark_df)
    print("Chart creation complete!")
except NameError:
    print("Error: benchmark_df not found. Please run the previous cell first.")

In [ ]:
def create_operation_tables(df):
    """Create a separate DataFrame table for each operation."""
    operation_tables = {}
    
    for operation in OPERATIONS:
        # Filter data for this operation
        op_data = df[df['operation'] == operation]
        if op_data.empty:
            print(f"No data for operation {operation}, skipping...")
            continue
            
        # Create a pivot table: message sizes in rows, schemes in columns
        pivot = op_data.pivot_table(
            index='msg_size',
            columns='display_name', 
            values='mean_ms',
            aggfunc='first'  # Just take the first value since each combo should be unique
        )
        
        # Sort the index (message sizes)
        pivot = pivot.sort_index()
        
        # Reorder columns based on SCHEME_NAMES order
        ordered_columns = [SCHEME_NAMES[scheme] for scheme in SCHEME_ORDER 
                          if SCHEME_NAMES[scheme] in pivot.columns]
        pivot = pivot[ordered_columns]
        
        # Add to dictionary
        operation_tables[operation] = pivot
        
        # Display table
        print(f"\n{operation.capitalize()} Operation (time in ms):")
        display(pivot.style.format("{:.2f}"))  # Format to 2 decimal places
        
    return operation_tables

# Create and display tables after charts
operation_tables = create_operation_tables(benchmark_df)

In [ ]:
def create_enhanced_operation_tables(df):
    """Create tables with additional statistics comparing schemes."""
    operation_tables = {}
    
    for operation in OPERATIONS:
        # Filter data for this operation
        op_data = df[df['operation'] == operation]
        if op_data.empty:
            print(f"No data for operation {operation}, skipping...")
            continue
            
        # Create a pivot table with raw values
        pivot = op_data.pivot_table(
            index='msg_size',
            columns='display_name', 
            values='mean_ms',
            aggfunc='first'
        )
        
        # Sort the index
        pivot = pivot.sort_index()
        
        # Reorder columns based on SCHEME_NAMES order
        ordered_columns = [SCHEME_NAMES[scheme] for scheme in SCHEME_ORDER 
                          if SCHEME_NAMES[scheme] in pivot.columns]
        pivot = pivot[ordered_columns]
        
        # Add to dictionary
        operation_tables[operation] = pivot
        
        # Display raw timing table
        print(f"\n{operation.capitalize()} Operation (time in ms):")
        display(pivot.style.format("{:.2f}"))
        
        # Find fastest scheme for each message size
        fastest = pivot.idxmin(axis=1)
        
        # Create percentage comparison table (relative to fastest)
        percentage = pivot.copy()
        for idx in pivot.index:
            fastest_scheme = fastest[idx]
            fastest_time = pivot.loc[idx, fastest_scheme]
            for col in pivot.columns:
                percentage.loc[idx, col] = (pivot.loc[idx, col] / fastest_time - 1) * 100
                
        # Display percentage table
        print(f"\n{operation.capitalize()} Comparison (% slower than fastest):")
        display(percentage.style.format("{:.1f}%"))
        
    return operation_tables
operation_tables2 = create_enhanced_operation_tables(benchmark_df)

In [ ]:
def generate_latex_tables(df):
    """Generate LaTeX tables for each operation."""
    latex_tables = {}
    
    for operation in OPERATIONS:
        # Filter data for this operation
        op_data = df[df['operation'] == operation]
        if op_data.empty:
            print(f"No data for operation {operation}, skipping...")
            continue
            
        # Create a pivot table
        pivot = op_data.pivot_table(
            index='msg_size',
            columns='display_name', 
            values='mean_ms',
            aggfunc='first'
        )
        
        # Sort the index
        pivot = pivot.sort_index()
        
        # Reorder columns based on SCHEME_NAMES order
        ordered_columns = [SCHEME_NAMES[scheme] for scheme in SCHEME_ORDER 
                          if SCHEME_NAMES[scheme] in pivot.columns]
        pivot = pivot[ordered_columns]
        
        # Generate LaTeX table with customizations
        latex_code = pivot.to_latex(
            float_format="%.2f",  # Format to 2 decimal places
            bold_rows=True,       # Bold the row labels (message sizes)
            caption=f"Performance of {operation.capitalize()} Operation (time in ms)",
            label=f"tab:{operation}_performance",
            position="htbp"       # Standard LaTeX table positioning
        )
        
        # Apply additional LaTeX formatting
        latex_code = latex_code.replace('\\begin{table}', '\\begin{table}[htbp]\n\\centering')
        
        # Add midrule after header
        header_end = latex_code.find('\\\\', latex_code.find('\\toprule'))
        if header_end != -1:
            latex_code = latex_code[:header_end+2] + '\\midrule\n' + latex_code[header_end+2:]
        
        # Store the LaTeX code
        latex_tables[operation] = latex_code
        
        # Print the LaTeX code
        print(f"\nLaTeX Table for {operation.capitalize()} Operation:")
        print(latex_code)
        print("\n" + "-"*80 + "\n")
        
    return latex_tables

# Generate LaTeX tables
latex_tables = generate_latex_tables(benchmark_df)

# If you want to save to files
for operation, latex_code in latex_tables.items():
    with open(f"latex_table_{operation}.tex", "w") as f:
        f.write(latex_code)

In [ ]:
def create_operation_tables3(df):
    """Create a separate DataFrame table for each operation and summary tables."""
    operation_tables = {}
    
    # First, create individual operation tables
    for operation in OPERATIONS:
        # Filter data for this operation
        op_data = df[df['operation'] == operation]
        if op_data.empty:
            print(f"No data for operation {operation}, skipping...")
            continue
            
        # Create a pivot table: message sizes in rows, schemes in columns
        pivot = op_data.pivot_table(
            index='msg_size',
            columns='display_name', 
            values='mean_ms',
            aggfunc='first'  # Just take the first value since each combo should be unique
        )
        
        # Sort the index (message sizes)
        pivot = pivot.sort_index()
        
        # Reorder columns based on SCHEME_NAMES order
        ordered_columns = [SCHEME_NAMES[scheme] for scheme in SCHEME_ORDER 
                          if SCHEME_NAMES[scheme] in pivot.columns]
        pivot = pivot[ordered_columns]
        
        # Add to dictionary
        operation_tables[operation] = pivot
        
        # Display table
        print(f"\n{operation.capitalize()} Operation (time in ms):")
        display(pivot.style.format("{:.2f}"))
    
    # Create combined tables if we have the necessary operations
    if 'obtain' in operation_tables and 'issue' in operation_tables:
        # Create sum of obtain and issue
        obtain_issue_sum = operation_tables['obtain'] + operation_tables['issue']
        
        # Display combined table
        print("\nIssuing Phase Total (Obtain + Issue) (time in ms):")
        display(obtain_issue_sum.style.format("{:.2f}"))
        
        # Add to dictionary
        operation_tables['obtain_issue_sum'] = obtain_issue_sum
    
    if 'show' in operation_tables and 'verify' in operation_tables:
        # Create sum of show and verify
        show_verify_sum = operation_tables['show'] + operation_tables['verify']
        
        # Display combined table
        print("\nShowing Phase Total (Show + Verify) (time in ms):")
        display(show_verify_sum.style.format("{:.2f}"))
        
        # Add to dictionary
        operation_tables['show_verify_sum'] = show_verify_sum
    
    # Generate a total table if all operations are present
    if all(op in operation_tables for op in OPERATIONS):
        total_sum = sum(operation_tables[op] for op in OPERATIONS)
        
        # Display total table
        print("\nTotal Time Across All Operations (time in ms):")
        display(total_sum.style.format("{:.2f}"))
        
        # Add to dictionary
        operation_tables['total_sum'] = total_sum
        
    return operation_tables
operation_tables3 = create_operation_tables3(benchmark_df)

In [ ]:
def generate_combined_latex_table_with_summaries(df):
    """Generate a single LaTeX table with all operations combined, including summary rows."""
    
    # Create a list to hold the rows for the final table
    table_rows = []
    
    # Get available schemes in the preferred order
    available_schemes = [s for s in SCHEME_ORDER if s in df['scheme'].unique()]
    scheme_display_names = [SCHEME_NAMES[scheme] for scheme in available_schemes]
    
    # Create column headers string
    column_headers = "n & " + " & ".join(scheme_display_names)
    
    # Create operation tables dictionary to store the data for summaries
    operation_pivot_tables = {}
    
    # Process each operation
    for operation in OPERATIONS:
        # Filter data for this operation
        op_data = df[df['operation'] == operation]
        if op_data.empty:
            print(f"No data for operation {operation}, skipping...")
            continue
            
        # Create a pivot table
        pivot = op_data.pivot_table(
            index='msg_size',
            columns='display_name', 
            values='mean_ms',
            aggfunc='first'
        )
        
        # Store in dictionary for summary calculations
        operation_pivot_tables[operation] = pivot
        
        # Sort the index
        pivot = pivot.sort_index()
        
        # Reorder columns based on scheme_display_names
        ordered_columns = [name for name in scheme_display_names if name in pivot.columns]
        pivot = pivot[ordered_columns]
        
        # Find minimum value in each row to highlight
        min_vals = pivot.min(axis=1)
        
        # Add operation header (centered)
        table_rows.append(f"\\multicolumn{{{len(ordered_columns)+1}}}{{c}}{{\\textbf{{{operation.capitalize()}}}}}  \\\\")
        table_rows.append("\\midrule")
        
        # Add data rows for this operation
        for idx in pivot.index:
            row = [f"\\textbf{{{idx}}}"]
            
            for col in ordered_columns:
                val = pivot.loc[idx, col]
                # Bold if it's the minimum value
                if val == min_vals[idx]:
                    row.append(f"\\textbf{{{val:.2f}}}")
                else:
                    row.append(f"{val:.2f}")
                    
            table_rows.append(" & ".join(row) + " \\\\")
        
        # Add spacing between operations
        table_rows.append("\\midrule")
    
    # Add summary tables if we have the required operations
    if 'obtain' in operation_pivot_tables and 'issue' in operation_pivot_tables:
        # Calculate sum
        obtain_issue_sum = operation_pivot_tables['obtain'] + operation_pivot_tables['issue']
        
        # Reorder columns
        ordered_columns = [name for name in scheme_display_names if name in obtain_issue_sum.columns]
        obtain_issue_sum = obtain_issue_sum[ordered_columns]
        
        # Find minimum values
        min_vals = obtain_issue_sum.min(axis=1)
        
        # Add summary header
        table_rows.append(f"\\multicolumn{{{len(ordered_columns)+1}}}{{c}}{{\\textbf{{Issuing Phase Total (Obtain + Issue)}}}}  \\\\")
        table_rows.append("\\midrule")
        
        # Add data rows
        for idx in obtain_issue_sum.index:
            row = [f"\\textbf{{{idx}}}"]
            
            for col in ordered_columns:
                val = obtain_issue_sum.loc[idx, col]
                # Bold if it's the minimum value
                if val == min_vals[idx]:
                    row.append(f"\\textbf{{{val:.2f}}}")
                else:
                    row.append(f"{val:.2f}")
                    
            table_rows.append(" & ".join(row) + " \\\\")
        
        table_rows.append("\\midrule")
    
    if 'show' in operation_pivot_tables and 'verify' in operation_pivot_tables:
        # Calculate sum
        show_verify_sum = operation_pivot_tables['show'] + operation_pivot_tables['verify']
        
        # Reorder columns
        ordered_columns = [name for name in scheme_display_names if name in show_verify_sum.columns]
        show_verify_sum = show_verify_sum[ordered_columns]
        
        # Find minimum values
        min_vals = show_verify_sum.min(axis=1)
        
        # Add summary header
        table_rows.append(f"\\multicolumn{{{len(ordered_columns)+1}}}{{c}}{{\\textbf{{Verify Phase Total (Show + Verify)}}}}  \\\\")
        table_rows.append("\\midrule")
        
        # Add data rows
        for idx in show_verify_sum.index:
            row = [f"\\textbf{{{idx}}}"]
            
            for col in ordered_columns:
                val = show_verify_sum.loc[idx, col]
                # Bold if it's the minimum value
                if val == min_vals[idx]:
                    row.append(f"\\textbf{{{val:.2f}}}")
                else:
                    row.append(f"{val:.2f}")
                    
            table_rows.append(" & ".join(row) + " \\\\")
    
    # Calculate number of columns for table specification
    num_columns = len(scheme_display_names) + 1  # +1 for the 'n' column
    
    # Generate the complete LaTeX table
    latex_code = f"""\\begin{{table}}[htbp]
\\centering
\\caption{{Performance of Anonymous Credential Operations (time in ms)}}
\\label{{tab:anoncred_performance}}
\\begin{{tabular}}{{@{{}}p{{1.2cm}}*{{{len(scheme_display_names)}}}{{>{{\\centering\\arraybackslash}}p{{1.6cm}}}}@{{}}}}
\\toprule
{column_headers} \\\\
\\midrule
{chr(10).join(table_rows)}
\\bottomrule
\\end{{tabular}}
\\end{{table}}
"""
    
    print("LaTeX Table for Combined Operations with Summaries:")
    print(latex_code)
    
    # Save to file
    with open("combined_table_with_summaries.tex", "w") as f:
        f.write(latex_code)
    
    return latex_code
combined_table = generate_combined_latex_table_with_summaries(benchmark_df)